In [17]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd

In [18]:

per_child_phase_in = Reform.from_dict({
  "gov.contrib.congress.wyden_smith.per_child_actc_phase_in": {
    "2024-01-01.2100-12-31": True
  },
  "gov.irs.credits.ctc.refundable.individual_max": {
    "2024-01-01.2100-12-31": 1000000
  },
}, country_id="us")

lower_phase_in = Reform.from_dict({
  "gov.irs.credits.ctc.refundable.phase_in.threshold": {
    "2024-01-01.2100-12-31": 0
  },
  "gov.irs.credits.ctc.refundable.individual_max": {
    "2024-01-01.2100-12-31": 1000000
  },
}, country_id="us")


baseline = Microsimulation()
reformed_per_child = Microsimulation(reform=per_child_phase_in)
reformed_lower = Microsimulation(reform=lower_phase_in)


In [19]:
# Initialize results list
results = []

# Calculate impacts for each year from 2024 to 2033
for year in range(2024, 2034):
    baseline = Microsimulation()
    reformed_per_child = Microsimulation(reform=per_child_phase_in)
    reformed_lower = Microsimulation(reform=lower_phase_in)

    baseline_income = baseline.calculate("household_net_income", period=year)
    reformed_per_child_income = reformed_per_child.calculate("household_net_income", period=year)
    reformed_lower_income = reformed_lower.calculate("household_net_income", period=year)

    difference_per_child = (reformed_per_child_income - baseline_income).sum()
    difference_lower = (reformed_lower_income - baseline_income).sum()

    results.append({
        "year": year,
        "per_child_phase_in_impact": difference_per_child,
        "lower_phase_in_impact": difference_lower
    })

In [20]:
# Convert results to DataFrame
df = pd.DataFrame(results)

In [21]:
# Calculate total impact over 10 years
total_per_child = df["per_child_phase_in_impact"].sum()
total_lower = df["lower_phase_in_impact"].sum()

print(f"Total impact over 10 years for per-child phase-in: ${total_per_child:,.0f}")
print(f"Total impact over 10 years for lower phase-in: ${total_lower:,.0f}")

# Export results to CSV
df.to_csv("ctc_reform_impacts.csv", index=False)
print("Results exported to ctc_reform_impacts.csv")


Total impact over 10 years for per-child phase-in: $16,361,076,408
Total impact over 10 years for lower phase-in: $7,082,324,098
Results exported to ctc_reform_impacts.csv


In [22]:
# Create a bar chart
import plotly.graph_objects as go
from policyengine_core.charts import format_fig

# Define the color scheme
colors = {
    "per_child_phase_in": "#0066cc",  # Light blue
    "lower_phase_in": "#003366",  # Dark blue
}

reform_names = {
    "per_child_phase_in": "Per child phase-in",
    "lower_phase_in": "Lower phase-in start"
}

# Create the yearly impact bar chart
fig1 = go.Figure()

for reform in ["per_child_phase_in", "lower_phase_in"]:
    fig1.add_trace(go.Bar(
        x=df["year"],
        y=df[f"{reform}_impact"] / 1e9,  # Convert to billions
        name=reform_names[reform],
        marker_color=colors[reform],
        text=df[f"{reform}_impact"].apply(lambda x: f'${x/1e9:.1f}B'),
        textposition='outside'
    ))

fig1.update_layout(
    title="Yearly CTC Reform Impacts",
    xaxis_title="Year",
    yaxis_title="Impact ($ Billions)",
    barmode='group',
    legend_title_text='Reform Type'
)

# Ensure all years are shown on x-axis
fig1.update_xaxes(
    tickmode='linear',
    tick0=2024,
    dtick=1
)

fig1 = format_fig(fig1)

# Create the total impact bar chart
fig2 = go.Figure()

fig2.add_trace(go.Bar(
    x=[reform_names["per_child_phase_in"], reform_names["lower_phase_in"]],
    y=[total_per_child / 1e9, total_lower / 1e9],  # Convert to billions
    marker_color=[colors["per_child_phase_in"], colors["lower_phase_in"]],
    text=[f'${total_per_child/1e9:.1f}B', f'${total_lower/1e9:.1f}B'],
    textposition='outside'
))

fig2.update_layout(
    title="Total CTC Reform Impacts Over 10 Years",
    xaxis_title="Reform Type",
    yaxis_title="Total Impact ($ Billions)",
)

fig2 = format_fig(fig2)

# Display both charts
fig1.show()
fig2.show()